# Generate figures for publication

##  Tactile stimulation results

Example imaging trials during tactile stimulation of the limbs. The code generates a maximum projection image of the stimulus-triggered average (STA) and a background-subtracted, normalized trace (called DF/F) of a pre-defined ROI (region of interest). The code also outputs the STA as a TIFF stack, and the whole figure as a vectorized PDF.

### Trials:
 * RHL: right hindlimb
 * LHL: left hindlimb
 * RFL: right forelimb

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from neuroprocessing.imagingtrials import ImagingTrialsLoader
from neuroprocessing.pipeline import load_user_config
from neuroprocessing.plots_for_pub import (
    plot_montage_and_trace_pairs,
    plot_STA_img_and_dff,
)

# this will load 'default.json' config. modify this to point to your config file
config = load_user_config("default")

plt.style.use("arcadia_updated.mplstyle")


# Define ROIs for the LHL and RHL in the somatosensory cortex (SSC)
# note: this is defined in non-flipped coordinates!

ssc_LHL_roi = {
    "center": (125, 175),  # (x, y)
    "width": 80,
    "height": 80,
}
ssc_RHL_roi = {
    "center": (265, 175),  # (x, y)
    "width": 80,
    "height": 80,
}

ssc_RFL_roi = {
    "center": (335, 210),  # (x, y)
    "width": 50,
    "height": 50,
}

os.makedirs("figs", exist_ok=True)

### RHL trial (Figure 2B)
* ID: `2024-02-29/Zyla_5min_RHLstim_2son4soff_1pt25pctISO_1`

In [ ]:
imaging_trials = ImagingTrialsLoader(config["processed_data_dir"])
imaging_trials.filter(
    exp_dir="2024-02-29",
    limb="RHL",
    rec_time="5min",
)
trial = imaging_trials.filtered_trials[0]

img = plot_STA_img_and_dff(
    trial,
    ssc_RHL_roi,
    show_montage=False,
    flipLR=True,
    bleach_subtract=True,
    tiff_out="figs/STA_RHL.tif",
    pdf_out="figs/STA_RHL.pdf",
    ylim=[-0.01, 0.035],
    xlim=[-0.05, 6.05],
)

# add colorbar
cmap = mpl.cm.inferno
norm = mpl.colors.Normalize(vmin=0, vmax=1)
f_colorbar, ax_colorbar = plt.subplots(figsize=(7, 0.5))

cb1 = mpl.colorbar.ColorbarBase(ax_colorbar, cmap=cmap, norm=norm, orientation="horizontal")

cb1.set_label("dF/F (norm.)", fontname=plt.rcParams["font.sans-serif"][0], fontweight="medium")
cb1.ax.tick_params(labelfontfamily=plt.rcParams["font.monospace"][0])

f_colorbar.savefig("figs/STA_RHL_colorbar.pdf", bbox_inches="tight")

### LHL trial (Figure 2C)
* ID: `2024-02-29/Zyla_5min_LHLstim_2son4soff_1pt25pctISO_2`

In [ ]:
imaging_trials.reset_filter()

imaging_trials.filter(exp_dir="2024-02-29", limb="LHL", rec_time="5min", remainder=".*pt25pctISO_2")
trial = imaging_trials.filtered_trials[0]

plot_STA_img_and_dff(
    trial,
    ssc_LHL_roi,
    show_montage=False,
    flipLR=True,
    bleach_subtract=True,
    tiff_out="figs/STA_LHL.tif",
    pdf_out="figs/STA_LHL.pdf",
    ylim=[-0.01, 0.035],
    xlim=[-0.05, 6.05],
)

### RFL trial (Figure 2D)
* ID: `2024-02-21/Zyla_5min_RFLstim_2son4soff_1pt25pctISO_deeper_2`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(
    exp_dir="2024-02-21",
    limb="RFL",
    rec_time="5min",
)
trial = imaging_trials.filtered_trials[0]

plot_STA_img_and_dff(
    trial,
    ssc_RFL_roi,
    show_montage=False,
    flipLR=True,
    bleach_subtract=True,
    tiff_out="figs/STA_RFL.tif",
    pdf_out="figs/STA_RFL.pdf",
    ylim=[-0.01, 0.035],
    xlim=[-0.05, 6.05],
)

## Injection results

Example imaging trials following injections of capsaicin or histamine (and controls). The code generates a montage of whole-brain activity, and a background-subtracted, normalized trace (DF/F) of a the full brain or a pre-defined ROI. The code also outputs the whole-brain activity as a TIFF stack, and the whole figure as a vectorized PDF.

*Note: all figures were flipped horizontally in the publication to match the orientation of the brain in the atlas.*

### Trials:
 * Capsaicin trials: capsaicin injection (studying pain response)
   * Injection into different limbs
   * Injection into the same limb
   * Injection into the nape of the neck
 * Histamine: histamine injection (studying itch response)
   * Injection into right hindlimb (RHL) (two examples)
   * Injection into left hindlimb (LHL) (two examples)



In [ ]:
COLOR_SALINE = {"line": "#336699", "fill": "#73B5E3"}
COLOR_HIST = {"line": "#B77A18", "fill": "#F5E4BE"}
COLOR_CAPSAICIN = {"line": "#F28360", "fill": "#FCCEAE"}

#### Same limb trial (Figure 3B)
Injection of capsaicin and control into the same (left) hindlimb.

* IDs
    * Control: `2024-03-18/Zyla_15min_LHL_carrierinj_1pt25pctISO_1`
    * Capsaicin: `2024-03-18/Zyla_30min_LHL_capsacirinj_1pt25pctISO_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-03-18", limb="LHL", injection_type=".*inj")

plot_montage_and_trace_pairs(
    imaging_trials,
    [COLOR_SALINE, COLOR_CAPSAICIN],
    montage_grid_shape=None,
    trace_ylim=[-1.5, 4],
    trace_xlim=[-50, 1400],
    tiff_out="figs/same_limb_",
    pdf_out="figs/same_limb.pdf",
)

#### Different limbs trial (Figure 3C)
Injection of capsaicin and control into different hindlimbs (vehicle into LHL, capsaicin into RHL).

* IDs
    * Control: `2024-03-19/Zyla_15min_LHL_carrierinj_1pt5pctISO_1_1`
    * Capsaicin: `2024-03-19/Zyla_30min_RHL_40ugin10uL_1pt5pctISO_1_1`


In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(
    exp_dir="2024-03-19",
    limb="(L|R)HL$",
)

plot_montage_and_trace_pairs(
    imaging_trials,
    [COLOR_SALINE, COLOR_CAPSAICIN],
    montage_grid_shape=None,
    trace_ylim=[-1, 2],
    trace_xlim=[-50, 1400],
    tiff_out="figs/diff_limbs_",
    pdf_out="figs/diff_limbs.pdf",
)

#### Nape trial (Figure 3D)

Injection of capsaicin into the nape of the neck (both vehicle and capsaicin).

* IDs
    * Control: `2024-03-18/Zyla_15min_nape_carrierinj_1pt25pctISO_1`
    * Capsaicin: `2024-03-18/Zyla_30min_nape_40ugin10uLcapsacininj_1pt25pctISO_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-03-18", limb="nape", injection_type=".*inj")

nape_roi = {
    "center": (288, 266),  # (x, y)
    "width": 50,
    "height": 50,
}

plot_montage_and_trace_pairs(
    imaging_trials[-1::-1],  # reverse order to plot saline first
    [COLOR_SALINE, COLOR_CAPSAICIN],
    montage_grid_shape=None,
    trace_ylim=[-1.5, 0.5],
    trace_xlim=[-50, 500],
    trace_roi=nape_roi,
    tiff_out="figs/nape_",
    pdf_out="figs/nape.pdf",
)

### Histamine trials

#### LHL trial (Figure 4B)

* IDs
    * Control: `2024-02-29/Zyla_15min_LHL_salineinj_withpushbutton_1`
    * Histamine: `2024-02-29/Zyla_30min_LHL_50uL27MMHistinj_withpushbutton_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-02-29", limb="LHL$", injection_type=".*inj")

plot_montage_and_trace_pairs(
    imaging_trials[-1::-1],
    [COLOR_SALINE, COLOR_HIST],
    montage_grid_shape=None,
    trace_ylim=[-1, 1],
    trace_xlim=[-50, 1400],
    tiff_out="figs/hist_lhl20240229_",
    pdf_out="figs/hist_lhl20240229.pdf",
)

#### RHL trial (Figure 4C)

* IDs
    * Control: `2024-02-29/Zyla_15min_RHL_salineinj_withpushbutton_1`
    * Histamine: `2024-02-29/Zyla_30min_RHL_50uL27MMHistinj_withpushbutton_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-02-29", limb="RHL$", injection_type=".*inj")

plot_montage_and_trace_pairs(
    imaging_trials,
    [COLOR_SALINE, COLOR_HIST],
    montage_grid_shape=None,
    trace_ylim=[-1, 1],
    trace_xlim=[-50, 1400],
    tiff_out="figs/hist_rhl20240229_",
    pdf_out="figs/hist_rhl20240229.pdf",
)

#### RHL trial (another example, not shown in main text)

* IDs
    * Control: `2024-03-06/Zyla_15min_RHL_salineinj_1pt25pctISO_1`
    * Histamine: `2024-03-06/Zyla_30min_RHL_27mMhistinj_1pt25pctISO_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-03-06", limb="RHL$", injection_type=".*inj")


plot_montage_and_trace_pairs(
    imaging_trials[-1::-1],
    [COLOR_SALINE, COLOR_HIST],
    montage_grid_shape=None,
    trace_ylim=[-1, 1],
    trace_xlim=[-50, 1400],
    tiff_out="figs/hist_rhl20240306_",
    pdf_out="figs/hist_rhl20240306.pdf",
)

#### LHL trial (another example, not shown in main text)

* IDs
    * Control: `2024-03-06/Zyla_15min_LHL_salineinj_1pt75pctISO_1`
    * Histamine: `2024-03-06/Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1`

In [ ]:
imaging_trials.reset_filter()
imaging_trials.filter(exp_dir="2024-03-06", limb="LHL$", injection_type=".*inj")

plot_montage_and_trace_pairs(
    imaging_trials,
    [COLOR_SALINE, COLOR_HIST],
    montage_grid_shape=None,
    trace_ylim=[-1, 1],
    trace_xlim=[-50, 1400],
    tiff_out="figs/hist_lhl20240306_",
    pdf_out="figs/hist_lhl20240306.pdf",
)